# Web scraping 

In [1]:
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests

load_dotenv()

def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

## Teams

I will be using http://www.footballsquads.co.uk to get a list of all the teams that played in the last 3 seasons of the premier league

In [2]:
def get_teams_by_season(season_year):
    soup = get_soup(f"http://www.footballsquads.co.uk/eng/{season_year}-{season_year+1}/engprem.htm")
    main_div = soup.find(id='main')
    team_list = list()
    for team in main_div.find_all('h5'):
        team_list.append(team.text.strip().replace(' & ', ' and '))
    return team_list

In [3]:
unique_teams = set() # as a set to make each team unique

for season in ([2018,2019,2020]):
    unique_teams.update(get_teams_by_season(season))
    
teams_df = pd.DataFrame(unique_teams, columns=["team"])
teams_df

,team
0,Arsenal
1,Burnley
2,Leicester City
3,West Ham United
4,Crystal Palace
5,Cardiff City
6,Tottenham Hotspur
7,Huddersfield Town
8,Bournemouth
9,Chelsea


## Stadiums

I will use http://www.myfootygrounds.co.uk/AreaMap.asp?view=ENGLAND# to get stadiums for the teams

In [4]:
def get_stadiums():
    soup = get_soup('http://www.myfootygrounds.co.uk/AreaMap.asp?view=ENGLAND#')
    stadium_table = soup.find_all(class_='table table-bordered')[1]
    stadiums = []
    for tr in stadium_table:
        td = tr.find_all('td')
        row = [tr.text.strip().replace(' & ', ' and ') for tr in td] # for consistency
        stadiums.append(row)
    stadium_df = pd.DataFrame(stadiums, columns=["team", "stadium", "league"])
    return stadium_df

In [5]:
stadium_df = get_stadiums()
stadium_df

,team,stadium,league
0,None,None,None
1,Accrington Stanley,Crown Ground,League One
2,AFC Wimbledon,Kingsmeadow,League One
3,Arsenal,Emirates Stadium,Premier League
4,Aston Villa,Villa Park,Premier League
...,...,...,...
88,West Bromwich Albion,The Hawthornes,Championship
89,West Ham United,London Stadium,Premier League
90,Wigan Athletic,The JJB Stadium,League One
91,Wolverhampton Wanderers,Molineux,Premier League


left merge on teams to get the 25 teams and their stadium

In [7]:
team_and_stadium_df = teams_df.merge(stadium_df,how="left", left_on="team",right_on="team")
team_and_stadium_df[["team","stadium"]]

,team,stadium
0,Arsenal,Emirates Stadium
1,Burnley,Turf Moor
2,Leicester City,Walkers Stadium
3,West Ham United,London Stadium
4,Crystal Palace,Selhurst Park
5,Cardiff City,Cardiff City Stadium
6,Tottenham Hotspur,White Hart Lane
7,Huddersfield Town,McAlpine Stadium
8,Bournemouth,Dean Court
9,Chelsea,Stamford Bridge


## City and stadium capacity

I will use https://soccer.nbcsports.com/2018/08/06/where-are-all-20-premier-league-teams-located-2/ to get city location and stadium capacity

In [8]:
def get_stadium_cities():
    soup = get_soup('https://soccer.nbcsports.com/2018/08/06/where-are-all-20-premier-league-teams-located-2/')
    stadium_cities_scrape = soup.find(class_="entry-content")
    stadium_cities = []
    for p in stadium_cities_scrape.find_all('p'):

        if p.text[0:5] == 'City:' or p.text[0:5] == 'Town:':
            split_p = (list(p.stripped_strings))
            city = split_p[0].replace('\xa0',' ').replace(' & ',' and ').replace(' (North)','').replace(' (East)','').replace(' (West)','').replace(' (South)','').split(': ')
            split_again = split_p[1].replace(':\xa0',': ').split(' (')
            stadium = split_again[0].split(': ')
            capacity = split_again[1].replace(')','').split(', ')
            row = [stadium[1], city[1],capacity[1].replace(',','')]
            stadium_cities.append(row)

        else:
            continue
    df = pd.DataFrame(stadium_cities, columns=["stadium","city","capacity"])
    return df

In [9]:
stadium_cities_df = get_stadium_cities()
stadium_cities_df

,stadium,city,capacity
0,Emirates Stadium,London,60432
1,Dean Court,Bournemouth,11464
2,American Express Community Stadium,Brighton and Hove,30750
3,Turf Moor,Burnley,21401
4,Cardiff City Stadium,"Cardiff, Wales",33280
5,Stamford Bridge,London,41623
6,Selhurst Park,London,26309
7,Goodison Park,Liverpool,40569
8,Craven Cottage,London,25700
9,John Smith’s Stadium,Huddersfield,24129


In [10]:
df = team_and_stadium_df.merge(stadium_cities_df,how="left", on="stadium")
df = df[["team","stadium","city","capacity"]]
df

,team,stadium,city,capacity
0,Arsenal,Emirates Stadium,London,60432
1,Burnley,Turf Moor,Burnley,21401
2,Leicester City,Walkers Stadium,NaN,NaN
3,West Ham United,London Stadium,London,5700
4,Crystal Palace,Selhurst Park,London,26309
5,Cardiff City,Cardiff City Stadium,"Cardiff, Wales",33280
6,Tottenham Hotspur,White Hart Lane,London,62062
7,Huddersfield Town,McAlpine Stadium,NaN,NaN
8,Bournemouth,Dean Court,Bournemouth,11464
9,Chelsea,Stamford Bridge,London,41623


Not all were filled due to slightly different names as well as some teams not being in the website. alter names to match and add the other manually.

In [19]:
team_and_stadium_df.loc[team_and_stadium_df["team"].str.contains("Leicester City"),'stadium'] = "King Power Stadium"
team_and_stadium_df.loc[team_and_stadium_df["team"].str.contains("Southampton"),'stadium'] = "St Mary’s Stadium"
team_and_stadium_df.loc[team_and_stadium_df["team"].str.contains("Newcastle United"),'stadium'] = "St James’ Park"
team_and_stadium_df.loc[team_and_stadium_df["team"].str.contains("Huddersfield Town"),'stadium'] = "John Smith’s Stadium"
team_and_stadium_df.loc[team_and_stadium_df["team"].str.contains("Brighton and Hove Albion"),'stadium'] = "American Express Community Stadium"

stadium_cities_df.loc[stadium_cities_df["stadium"].str.contains("Etihad"),'stadium'] = "City of Manchester Stadium"
df = team_and_stadium_df.merge(stadium_cities_df,how="left", on="stadium")
df = df[["team","stadium","city","capacity"]]
df



,team,stadium,city,capacity
0,Arsenal,Emirates Stadium,London,60432
1,Burnley,Turf Moor,Burnley,21401
2,Leicester City,King Power Stadium,Leicester,32500
3,West Ham United,London Stadium,London,5700
4,Crystal Palace,Selhurst Park,London,26309
5,Cardiff City,Cardiff City Stadium,"Cardiff, Wales",33280
6,Tottenham Hotspur,White Hart Lane,London,62062
7,Huddersfield Town,John Smith’s Stadium,Huddersfield,24129
8,Bournemouth,Dean Court,Bournemouth,11464
9,Chelsea,Stamford Bridge,London,41623


There are now just 5 missing stadium details which I have added manually.

team | stadium | city | capacity
------------ | ------------- | ------------- | -------------
West Bromwich Albion | The Hawthornes  | West Bromwich |26688
Aston Villa |Villa Park | Birmingham | 42682
Leeds United | Elland Road | Leeds | 37792
Sheffield United | Bramall Lane | Sheffield | 32050
Norwich City | Carrow Road | Norwich | 27359

In [22]:
df.loc[df["team"].str.contains("West Bromwich Albion"), ["city","capacity"]] = [["West Bromwich", 26688]]
df.loc[df["team"].str.contains("Aston Villa"), ["city","capacity"]] = [["Birmingham", 42682]]
df.loc[df["team"].str.contains("Leeds United"), ["city","capacity"]] = [["Leeds", 37792]]
df.loc[df["team"].str.contains("Sheffield United"), ["city","capacity"]] = [["Sheffield", 32050]]
df.loc[df["team"].str.contains("Norwich City"), ["city","capacity"]] = [["Norwich", 27359]]

missing values now filled:

In [23]:
df

,team,stadium,city,capacity
0,Arsenal,Emirates Stadium,London,60432
1,Burnley,Turf Moor,Burnley,21401
2,Leicester City,King Power Stadium,Leicester,32500
3,West Ham United,London Stadium,London,5700
4,Crystal Palace,Selhurst Park,London,26309
5,Cardiff City,Cardiff City Stadium,"Cardiff, Wales",33280
6,Tottenham Hotspur,White Hart Lane,London,62062
7,Huddersfield Town,John Smith’s Stadium,Huddersfield,24129
8,Bournemouth,Dean Court,Bournemouth,11464
9,Chelsea,Stamford Bridge,London,41623


## Lat/long

i will use this api to get the lat/long of cities : https://developer.mapquest.com/documentation/open/geocoding-api/

In [24]:
def get_lat_long(city):
    if city == 'Cardiff':
        country = 'Wales'
    else:
        country = 'UK'

    response = requests.get(f"http://open.mapquestapi.com/geocoding/v1/address?key={os.getenv('MAPQUEST_KEY')}&location={city},{country}")
    data = response.json()
    lat_lng = data['results'][0]['locations'][0]['latLng']
    return lat_lng

In [25]:
df['lat_long'] = df["city"].apply(lambda x: get_lat_long(x))
df['lat'] = df['lat_long'].apply(lambda x: x['lat'])
df['lng'] = df['lat_long'].apply(lambda x: x['lng'])
df = df[["team","stadium","city","capacity","lat","lng"]]

df

,team,stadium,city,capacity,lat,lng
0,Arsenal,Emirates Stadium,London,60432,51.507322,-0.127647
1,Burnley,Turf Moor,Burnley,21401,53.790726,-2.243920
2,Leicester City,King Power Stadium,Leicester,32500,52.636140,-1.133079
3,West Ham United,London Stadium,London,5700,51.507322,-0.127647
4,Crystal Palace,Selhurst Park,London,26309,51.507322,-0.127647
5,Cardiff City,Cardiff City Stadium,"Cardiff, Wales",33280,51.481655,-3.179193
6,Tottenham Hotspur,White Hart Lane,London,62062,51.507322,-0.127647
7,Huddersfield Town,John Smith’s Stadium,Huddersfield,24129,53.646665,-1.782248
8,Bournemouth,Dean Court,Bournemouth,11464,50.744670,-1.857981
9,Chelsea,Stamford Bridge,London,41623,51.507322,-0.127647
